### dependency 설치

In [ ]:
%pip install -q llama-index
%pip install llama-index-llms-gemini
%pip install -q llama-index google-generativeai
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

# llama index를 사용해 gemini api 불러오기

### pdf 불러오기

In [3]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
doc1 = reader.load_data()

# doc1 = SimpleDirectoryReader(input_files=["data"]).load_data()

### 임베딩 다운로드

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_ko = HuggingFaceEmbedding(model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr") 

### llama index 설정

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from llama_index.core import ServiceContext

llm = Gemini(model_name='models/gemini-1.5-flash', request_timeout=120.0)

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20, embed_model=embed_model_ko)
index = VectorStoreIndex.from_documents(doc1,service_context=service_context,show_progress=True)

index.storage_context.persist()

query_engine = index.as_query_engine()


### 질문과 답변 출력

In [7]:
response = query_engine.query("항만에 대해 정확한 정의를 설명해봐")

print(response)

response = query_engine.query("해운업에 대한 정의를 말해봐")

print(response)

항만은 선박의 출입, 사람의 승선ㆍ하선, 화물의 하역ㆍ보관 및 처리, 해양친수활동 등을 위한 시설과 화물의 조립ㆍ가공ㆍ포장ㆍ제조 등 부가가치 창출을 위한 시설이 갖추어진 곳을 말합니다. 

해운업은 해상여객운송사업, 해상화물운송사업, 해운중개업, 해운대리점업, 선박대여업 및 선박관리업을 말합니다. 

